In [1]:
!pip install wget
!pip install wandb
!pip install xtarfile

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=78c126e79260f1c0414096cac929983828e7cbb950a7d7e71d7aa3a63eccca9e
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
     |████████████████████████████████| 1.8 MB 7.1 MB/s 
     |████████████████████████████████| 144 kB 67.8 MB/s 
     |████████████████████████████████| 181 kB 73.2 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=19b4bdee1ae8af9af1de922b3e93a6a2e41dd14fef2b84b77c21ce3fb9ac7c26
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
  Created wheel for xtarfile: filename=xtarfile-0.1.0-py3-none-any.whl size=3862 sha256=0e4cebc609a269d6da4e41deb0c3e8b11458650dcf47634dcc764d137cf0c4eb
  Stored in directory: /root/.cache/pip/wheels/14/ed/

In [2]:
import wandb

### Configuration

In [3]:
START_TOKEN="0"
END_TOKEN="1"

In [4]:

import os
from os.path import exists
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random 
import numpy as np
import keras
import tensorflow as tf
import xtarfile as tarfile
import csv
class Dataset:
    
    def downloadDataSet(self):
      cwd = os.getcwd()
     
      file_exists = exists('./dakshina_dataset_v1.0.tar')
      if(file_exists==False):
        print('downloading....')
        os.system('curl -SL https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar > dakshina_dataset_v1.0.tar')
        print('download Complete')
      extract_exists = exists('./dakshina_dataset_v1.0/')   
      if(extract_exists==False): 
        print('Extracting..') 
        with tarfile.open('dakshina_dataset_v1.0.tar', 'r') as archive:
            archive.extractall()
        print('Complete')

      print('You are all set')

    def readData(self,dir,start_token="0",end_token="1"):
      # Vectorize the data.
      input_texts=list()
      target_texts=list()
      original_input_texts=list()
      original_target_texts=list()
      input_characters = set()
      target_characters = set()
      with open(dir, "r", encoding="utf-8") as f:
        lines = f.read().split("\n")
      
      for line in lines:
          ln = line.split("\t")
          if(len(ln)==3):
            target_text=ln[0].strip()  
            input_text=ln[1].strip()
            original_input_texts.append(input_text)
            original_target_texts.append(target_text)
            input_text+=end_token
            target_text = start_token + target_text + end_token
            input_texts.append(input_text)
            target_texts.append(target_text)
            for char in input_text:
              if char not in input_characters:
                input_characters.add(char)
            for char in target_text:
             if char not in target_characters:
               target_characters.add(char)
      input_characters = sorted(list(input_characters))
      target_characters = sorted(list(target_characters))
      return original_input_texts,original_target_texts,input_texts,target_texts,input_characters,target_characters


   

    #def encode_decode_characters(self,train_input, train_target, val_input=[], val_target=[],start_token="\t",end_token="\n"):
    def encode_decode_characters(self,train_input, train_target,start_token="0",end_token="1"):
        # Returns the encoding for characters to integer (as a dictionary) and decoding for integers to characters (as a list) for in
        BLANK_CHAR=' '
        # Encoding and decoding of input vocabulary
        input_char_enc = {}
        input_char_dec = []
        max_encoder_seq_length = 1
        tmplist=[]
        for i in train_input:
            tmplist.append(i)
        """for i in val_input:
            tmplist.append(i)"""
        for string in tmplist:
            max_encoder_seq_length = max(max_encoder_seq_length, len(string))
            for char in string:
                if char not in input_char_enc:
                    input_char_enc[char] = len(input_char_dec)
                    input_char_dec.append(char)
        """if BLANK_CHAR not in input_char_enc:
            input_char_enc[BLANK_CHAR] = len(input_char_dec)
            input_char_dec.append(BLANK_CHAR)
"""
        # Encoding and decoding of target vocabulary
        target_char_enc = {}
        target_char_dec = []
        #target_char_enc[start_token] = len(target_char_dec)
        #target_char_dec.append(start_token)
        max_decoder_seq_length = 1
        for string in train_target :#+ val_target:
            max_decoder_seq_length = max(max_decoder_seq_length, len(string)+2)
            for char in string:
                if char not in target_char_enc:
                    target_char_enc[char] = len(target_char_dec)
                    target_char_dec.append(char)
        #target_char_enc[end_token] = len(target_char_dec)
        #target_char_dec.append(end_token)
        """if ' ' not in target_char_enc:
            target_char_enc[BLANK_CHAR] = len(target_char_dec)
            target_char_dec.append(BLANK_CHAR)"""

        """print("Number of training samples:", len(train_input))
        print("Number of validation samples:", len(val_input))
        print("Number of unique input tokens:", len(input_char_dec))
        print("Number of unique output tokens:", len(target_char_dec))
        print("Max sequence length for inputs:", max_encoder_seq_length)
        print("Max sequence length for outputs:", max_decoder_seq_length)"""

        return input_char_enc, input_char_dec, target_char_enc, target_char_dec, max_encoder_seq_length, max_decoder_seq_length
    def preprocessing(self,input_char_enc, input_char_dec, target_char_enc, target_char_dec,max_encoder_seq_length,max_decoder_seq_length,input_texts,target_texts,start_token="0",end_token="1"):
        encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length), dtype="float32")
        decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length), dtype="float32")
        decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, len(target_char_dec)), dtype="float32")
        for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
            for t, char in enumerate(input_text):
                encoder_input_data[i, t] =input_char_enc[char]
            #encoder_input_data[i, t + 1 :] = input_char_enc[start_token]

            for t, char in enumerate(target_text):
                # decoder_target_data is ahead of decoder_input_data by one timestep
                decoder_input_data[i, t, ] = target_char_enc[char]
                if t > 0:
                    # decoder_target_data will be ahead by one timestep
                    # and will not include the start character.
                    decoder_target_data[i, t - 1, target_char_enc[char]] = 1.0
            decoder_input_data[i, t + 1 :] = target_char_enc[end_token]
            decoder_target_data[i, t:, target_char_enc[end_token]] = 1.0
        return encoder_input_data,decoder_input_data,decoder_target_data


       
   

In [5]:
WANDB_LOG=True

### Sequence to sequence RNN

In [6]:
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.utils.vis_utils import plot_model
class seqTOseq:
  def parameters(self,input_char_dec,target_char_dec,\
                 num_of_encoders=1,\
                 num_of_decoders=1,\
                 latent_dim=64,\
                 inp_emb_size=64,\
                 cell_type="lstm",\
                 cell_activation="tanh",\
                 optimizer="adam",\
                 dropout=0,\
                 lr=0.001):
      self.num_encoder_tokens=len(input_char_dec)
      self.num_decoder_tokens=len(target_char_dec)
      self.latent_dim=latent_dim
      self.cell_type=cell_type
      self.optimizer=optimizer
      self.num_of_encoders=num_of_encoders
      self.num_of_decoders=num_of_decoders
      self.dropout=dropout
      self.inp_emb_size=inp_emb_size
      self.cell_activation=cell_activation
      self.lr=lr
  def model(self):
      if(self.cell_type=="rnn"):
         cell=keras.layers.SimpleRNN
      if(self.cell_type=="lstm"):
        cell=keras.layers.LSTM     
      if(self.cell_type=="gru"):
         cell=keras.layers.GRU
      # Define an input sequence and process it.
      
      encoder_input = keras.layers.Input(shape=(None,), name="input_1")
      encoder_inp_emb = keras.layers.Embedding(self.num_encoder_tokens, self.inp_emb_size, name="embedding_1")(encoder_input)
      encoder =cell(self.latent_dim, return_state=True,\
                    return_sequences=True,\
                    activation=self.cell_activation,\
                    name="encoder_1",\
                    dropout=self.dropout)        
      
      encoder_outputs, *encoder_states = encoder(encoder_inp_emb)
      for i in range(1,self.num_of_encoders):
        encoder_outputs, *encoder_states = cell(self.latent_dim,\
                                       return_state=True,\
                                       return_sequences=True,\
                                       activation=self.cell_activation,\
                                       dropout=self.dropout,\
                                       name="encoder_"+str(i+1))(
                                                        encoder_outputs,initial_state=encoder_states
                                                 )
      # Set up the decoder, using `encoder_states` as initial state.
      decoder_inputs = keras.Input(shape=(None,), name="input_2")
      #input embedding 
      decoder_inp_emb = keras.layers.Embedding(self.num_decoder_tokens, self.inp_emb_size, name="embedding_2")(decoder_inputs)
      decoder=cell(self.latent_dim ,return_sequences=True,\
                   return_state=True,\
                   name="decoder_1",\
                   activation=self.cell_activation,\
                   dropout=self.dropout)
      decoder_outputs, *decode_states=decoder(decoder_inp_emb, initial_state=encoder_states)
      for i in range(1,self.num_of_decoders):
        decoder_outputs, *decode_states = cell(self.latent_dim,\
                                       return_state=True,\
                                       return_sequences=True,\
                                       activation=self.cell_activation,\
                                       dropout=self.dropout,\
                                       name="decoder_"+str(i+1))(
                                                        decoder_outputs,initial_state=encoder_states)
                                                 
      decoder_dense = keras.layers.Dense(self.num_decoder_tokens, activation="softmax")
      decoder_outputs = decoder_dense(decoder_outputs)
      # Define the model that will turn
      # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
      model = keras.Model([encoder_input, decoder_inputs], decoder_outputs)
      if(self.optimizer=="rmsprop"):
        optimizer=tf.keras.optimizers.RMSprop(learning_rate=self.lr)
      else:
        optimizer=tf.keras.optimizers.Adam(learning_rate=self.lr)
      model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
      return model
  def modelSummary(self,model):
      plot_model(model, to_file='encoder_model.png', show_shapes=True)
      f = plt.figure()
      f.set_figwidth(20)
      f.set_figheight(20)
      plt.imshow(mpimg.imread('encoder_model.png'))
      plt.axis('off')
 

  

### Read data from dataset

In [7]:
ds=Dataset()
ds.downloadDataSet()
language='te'
train_dir='./dakshina_dataset_v1.0/'+language+'/lexicons/'+language+'.translit.sampled.train.tsv'
val_dir='./dakshina_dataset_v1.0/'+language+'/lexicons/'+language+'.translit.sampled.dev.tsv'
test_dir='./dakshina_dataset_v1.0/'+language+'/lexicons/'+language+'.translit.sampled.test.tsv'


downloading....
download Complete
Extracting..
Complete
You are all set


### Preprocessing data

In [8]:
train_original_input_texts,train_original_target_texts,train_ip_txt,train_op_txt,train_ip_chr,train_op_chr=ds.readData(train_dir)
val_original_input_texts,val_original_target_texts,val_ip_txt,val_op_txt,val_ip_chr,val_op_chr=ds.readData(val_dir)
test_original_input_texts,test_original_target_texts,test_ip_txt,test_op_txt,test_ip_chr,test_op_chr=ds.readData(test_dir)

In [9]:
"""input_char_enc, input_char_dec,\
 target_char_enc,\
target_char_dec,\
max_encoder_seq_length,\
max_decoder_seq_length\
=ds.encode_decode_characters(train_ip_txt, train_op_txt, val_ip_txt, val_op_txt)"""

input_char_enc, input_char_dec,\
target_char_enc,\
target_char_dec,\
max_encoder_seq_length,\
max_decoder_seq_length\
=ds.encode_decode_characters(train_ip_txt, train_op_txt)

##### Assigning training, validation and test encoder input, decoder input, decoder output

In [10]:

train_enc_input, train_dec_input, train_dec_target\
= ds.preprocessing(input_char_enc, input_char_dec, target_char_enc, target_char_dec,max_encoder_seq_length,max_decoder_seq_length,train_ip_txt, train_op_txt)



In [11]:
val_enc_input, val_dec_input, val_dec_target = \
ds.preprocessing(input_char_enc, input_char_dec, target_char_enc, target_char_dec,\
                 max_encoder_seq_length,max_decoder_seq_length,val_ip_txt, val_op_txt)


test_enc_input, test_dec_input, test_dec_target =\
 ds.preprocessing(input_char_enc, input_char_dec, target_char_enc, target_char_dec,max_encoder_seq_length,max_decoder_seq_length\
                  ,test_ip_txt, test_op_txt)


In [12]:
class CustomCallback(keras.callbacks.Callback):
    def __init__(self, model):
        self.model = model
       
    def on_epoch_end(self, epoch, logs={}):
        key_list = list(target_char_enc.keys())
        val_list = list(target_char_enc.values())
        predicted_data=self.model.predict([val_enc_input, val_dec_input ])
        targetpredicted=[]
        itr=0
        for data in predicted_data:
          wordip=test_ip_txt[itr]
          itr+=1
          wordop=""
          for i in data:
          
            index=np.argmax(i)
            position = val_list.index(index)
            wordop+=key_list[position]

          #wordop=wordop.replace('\t', '')
          #wordop=wordop.replace('\n', '')
          wordop=wordop.replace('0', '')
          wordop=wordop.replace('1', '')
          #wordop=wordop.replace(' ', '')
          targetpredicted.append(wordop)
          count=0
          countTot=0
        for i in range(len(targetpredicted)):
          countTot+=1
          if(val_original_target_texts[i]==targetpredicted[i]):
              count+=1
        print(" val_exact_accuracy: "+str (count/countTot))
        if(WANDB_LOG):
          cnt=count/countTot
          wandb.log({"val_exact_accuracy":cnt})
        

In [13]:

from wandb.keras import WandbCallback

#### Train the model

In [14]:
sweep_config = {
  "name": "DL_Assignment3_Rnn",
  "method": "bayes",
  "metric": {
      "name": "val_exact_accuracy",
      "goal": "maximize",
  },
  
  "parameters": {
        "num_of_encoders":{
          "values":[1,2,3]    
        },
        "num_of_decoders":{
            "values":[1,2,3]      

        },
        "cell_type":{
          "values":['lstm','gru','rnn']  
        },
        
        "lr":{
          "values":[0.01,0.001,0.0001]  
        },
        "optimizer":{
          "values":['adam','rmsprop']  
        },
        "dropout":{ "values": [0.25,0.3,0.4,0.5]},
        "latent_dim":{ "values": [64,128,256,512]},
        "inp_emb_size": {"values": [64,128,256]},
        "batch_size":{"values":[32,64,128,256]}
        }
    }

In [15]:


# This is the main function to use to train/fine-tune the model using wandb runs
def train_wandb():
    run = wandb.init()
  
    config=wandb.config
    # Set the run name
    name="num_of_encoders("+ str(config["num_of_encoders"]) + ")_"
    name = " num_of_decoders(" + str(config["num_of_decoders"]) + ")_"
    name += " cell_type(" + str(config["cell_type"]) + ")_"
    name += "latent_dim(" + str(config["latent_dim"])+ ")_"
    name += "lr(" + str(config["lr"])+ ")_"
    name += "optimizer(" + str(config["optimizer"]) + ")_"
    name += "dropout(" + str(config["dropout"]) + ")"
    name += "inp_emb_size(" + str(config["inp_emb_size"]) + ")_"
    name+="batch_size(" + str(config["batch_size"]) + ")"


    wandb.run.name = name[:-1]
    batch_size=config["batch_size"]
    inp_emb_size=config["inp_emb_size"]
    dropout=config["dropout"]
    optimizer=config["optimizer"]

    num_of_encoders=config["num_of_encoders"]
    num_of_decoders=config["num_of_decoders"]

    lr=config["lr"]
    latent_dim=config["latent_dim"]
    cell_type=config["cell_type"]

    
    seq=seqTOseq()
    seq.parameters(input_char_dec=input_char_dec,\
                 target_char_dec=target_char_dec,\
                 num_of_encoders=num_of_encoders,\
                 num_of_decoders=num_of_decoders,
                 latent_dim=latent_dim,\
                 inp_emb_size=inp_emb_size,\
                 cell_type=cell_type,\
                 cell_activation="tanh",\
                 optimizer=optimizer,\
                 dropout=dropout,\
                 lr=lr)

    model=seq.model()
    earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3,restore_best_weights=True)
    model.fit(
      [train_enc_input, train_dec_input],
      train_dec_target,
      batch_size=batch_size,
      epochs=25,
     validation_data=([val_enc_input, val_dec_input ] ,val_dec_target),
     callbacks=[earlystop,CustomCallback(model),WandbCallback()]
    )



In [16]:
    

sweep_id = wandb.sweep(sweep_config, entity="kankan-jana", project="CS6910_Assignment3")


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: 2y26bj6j
Sweep URL: https://wandb.ai/kankan-jana/CS6910_Assignment3/sweeps/2y26bj6j


In [17]:


wandb.agent(sweep_id, train_wandb)

wandb: Agent Starting Run: xtikkftz with config:
wandb: 	batch_size: 64
wandb: 	cell_type: lstm
wandb: 	dropout: 0.25
wandb: 	inp_emb_size: 256
wandb: 	latent_dim: 256
wandb: 	lr: 0.01
wandb: 	num_of_decoders: 1
wandb: 	num_of_encoders: 1
wandb: 	optimizer: adam
wandb: Currently logged in as: kankan-jana (use `wandb login --relogin` to force relogin)


Epoch 1/25
915/915 [==============================] - 18s 13ms/step - loss: 0.7724 - accuracy: 0.7819 - val_loss: 0.5886 - val_accuracy: 0.8308 - _timestamp: 1651409633.0000 - _runtime: 26.0000
Epoch 2/25
915/915 [==============================] - 10s 11ms/step - loss: 0.4837 - accuracy: 0.8579 - val_loss: 0.4710 - val_accuracy: 0.8657 - _timestamp: 1651409643.0000 - _runtime: 36.0000
Epoch 3/25
915/915 [==============================] - 10s 11ms/step - loss: 0.4048 - accuracy: 0.8787 - val_loss: 0.4434 - val_accuracy: 0.8710 - _timestamp: 1651409653.0000 - _runtime: 46.0000
Epoch 4/25
915/915 [==============================] - 10s 11ms/step - loss: 0.3657 - accuracy: 0.8890 - val_loss: 0.4170 - val_accuracy: 0.8803 - _timestamp: 1651409663.0000 - _runtime: 56.0000
Epoch 5/25
915/915 [==============================] - 10s 11ms/step - loss: 0.3295 - accuracy: 0.8986 - val_loss: 0.3824 - val_accuracy: 0.8886 - _timestamp: 1651409673.0000 - _runtime: 66.0000
Epoch 6/25
915/915 [==========

accuracy,▁▅▆▇▇████████
epoch,▁▂▂▃▃▄▅▅▆▆▇▇█
loss,█▄▃▂▂▁▁▁▁▁▁▁▁
val_accuracy,▁▅▅▆▇▇▇██████
val_exact_accuracy,▁▂▂▃▅▆▆▇██▇▇█
val_loss,█▅▄▃▂▁▁▁▁▁▁▁▁
accuracy,0.91355
best_epoch,9
best_val_loss,0.35319
epoch,12
loss,0.27548


wandb: Agent Starting Run: dbfp56gp with config:
wandb: 	batch_size: 64
wandb: 	cell_type: lstm
wandb: 	dropout: 0.5
wandb: 	inp_emb_size: 128
wandb: 	latent_dim: 512
wandb: 	lr: 0.001
wandb: 	num_of_decoders: 1
wandb: 	num_of_encoders: 2
wandb: 	optimizer: adam


Epoch 1/25
915/915 [==============================] - 27s 25ms/step - loss: 0.9504 - accuracy: 0.7390 - val_loss: 0.6731 - val_accuracy: 0.8084 - _timestamp: 1651409801.0000 - _runtime: 31.0000
Epoch 2/25
915/915 [==============================] - 21s 23ms/step - loss: 0.4705 - accuracy: 0.8621 - val_loss: 0.2420 - val_accuracy: 0.9271 - _timestamp: 1651409822.0000 - _runtime: 52.0000
Epoch 3/25
915/915 [==============================] - 21s 23ms/step - loss: 0.1898 - accuracy: 0.9436 - val_loss: 0.1355 - val_accuracy: 0.9595 - _timestamp: 1651409843.0000 - _runtime: 73.0000
Epoch 4/25
915/915 [==============================] - 21s 23ms/step - loss: 0.1069 - accuracy: 0.9682 - val_loss: 0.1132 - val_accuracy: 0.9661 - _timestamp: 1651409864.0000 - _runtime: 94.0000
Epoch 5/25
915/915 [==============================] - 21s 23ms/step - loss: 0.0741 - accuracy: 0.9779 - val_loss: 0.1041 - val_accuracy: 0.9685 - _timestamp: 1651409885.0000 - _runtime: 115.0000
Epoch 6/25
915/915 [=========

...

wandb: Agent Starting Run: 0giz3aui with config:
wandb: 	batch_size: 32
wandb: 	cell_type: lstm
wandb: 	dropout: 0.5
wandb: 	inp_emb_size: 256
wandb: 	latent_dim: 512
wandb: 	lr: 0.0001
wandb: 	num_of_decoders: 2
wandb: 	num_of_encoders: 2
wandb: 	optimizer: rmsprop


Epoch 1/25
1830/1830 [==============================] - 58s 29ms/step - loss: 1.0784 - accuracy: 0.7121 - val_loss: 0.8202 - val_accuracy: 0.7696 - _timestamp: 1651474342.0000 - _runtime: 67.0000
Epoch 2/25
1830/1830 [==============================] - 50s 27ms/step - loss: 0.7951 - accuracy: 0.7738 - val_loss: 0.6631 - val_accuracy: 0.8117 - _timestamp: 1651474392.0000 - _runtime: 117.0000
Epoch 3/25
1026/1830 [===============>..............] - ETA: 20s - loss: 0.6722 - accuracy: 0.8057

wandb: ERROR Error while calling W&B API: context deadline exceeded (<Response [500]>)


1830/1830 [==============================] - 50s 27ms/step - loss: 0.6455 - accuracy: 0.8124 - val_loss: 0.5320 - val_accuracy: 0.8447 - _timestamp: 1651474442.0000 - _runtime: 167.0000
Epoch 4/25
1830/1830 [==============================] - 50s 27ms/step - loss: 0.5267 - accuracy: 0.8439 - val_loss: 0.4336 - val_accuracy: 0.8695 - _timestamp: 1651474491.0000 - _runtime: 216.0000
Epoch 5/25
1830/1830 [==============================] - 50s 27ms/step - loss: 0.4413 - accuracy: 0.8676 - val_loss: 0.3739 - val_accuracy: 0.8860 - _timestamp: 1651474541.0000 - _runtime: 266.0000
Epoch 6/25
1830/1830 [==============================] - 50s 27ms/step - loss: 0.3754 - accuracy: 0.8866 - val_loss: 0.3176 - val_accuracy: 0.9031 - _timestamp: 1651474591.0000 - _runtime: 316.0000
Epoch 7/25
1830/1830 [==============================] - 50s 27ms/step - loss: 0.3190 - accuracy: 0.9031 - val_loss: 0.2761 - val_accuracy: 0.9148 - _timestamp: 1651474641.0000 - _runtime: 366.0000
Epoch 8/25
1830/1830 [====

wandb: Ctrl + C detected. Stopping sweep.
